- Pradeep Gahlawat
- 10524011
- Driver drowsiness detection system
- file 1/4 (Data collection)

In [1]:
# import required libraries
import numpy as np
import pandas as pd 
from imutils import face_utils
import math
import dlib
import cv2
from scipy.spatial import distance

In [2]:
# This cell perform model preperation steps.
## first it loads the video into an opencv instance
### then it converts these videos into images
#### after that it detects faces using dlib's  HOG +SVM to detector
##### then again using dlib's pre-trained 68 landmarks model detects 68 2d coordinates.
#### image is again used to calculate eulers angles for head pose

p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

j = 40  # folder
i = 0  # file

# load video into a cv2 instance
cap = cv2.VideoCapture('C:/Users/Pradeep/Downloads/Colab/Videos/' + str(j) +'/' + str(i) + '.mp4')

data =[]
labels = [] 
rotation_degree =[]

count= 0
sec = 0
import time
start_time = time.time()
while True and count < 240:    
    # Set startting position at three minutes
    cap.set(cv2.CAP_PROP_POS_MSEC, 180000 + sec*1000)    
    # Getting frames from the video 
    _, image = cap.read()
    
    # Rotating image to change orientation
#     image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE) #####  Uncomment this line to change rotation in case code fails. 
   
    # Converting the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    
    # Get faces into webcam's image
    rects = detector(gray, 0)    
    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        size = image.shape
        print(count)
        image_points = np.array([
                            (shape[33, :]),     # Nose tip
                            (shape[8,  :]),     # Chin
                            (shape[36, :]),     # Left eye left corner
                            (shape[45, :]),     # Right eye right corne
                            (shape[48, :]),     # Left Mouth corner
                            (shape[54, :])      # Right mouth corner
                        ], dtype="double")

        # 3D model points in word coordinates.
        model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner                     
                        ])
       
        # Camera internals
        focal_length = size[1]
        center = (size[1]/2, size[0]/2)
        camera_matrix = np.array(
                                [[focal_length, 0, center[0]],
                                [0, focal_length, center[1]],
                                [0, 0, 1]], dtype = "double"
                                )
        

        dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
        # estimating pose using solvepnp function
        (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
        
        #get euler angles using rotation and translation vectors
        rvec_matrix = cv2.Rodrigues(rotation_vector)[0]
        proj_matrix = np.hstack((rvec_matrix, translation_vector))
        eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6] 
        pitch, yaw, roll = [math.radians(angle) for angle in eulerAngles]
        pitch = math.degrees(math.asin(math.sin(pitch)))
        roll = -math.degrees(math.asin(math.sin(roll)))
        yaw = math.degrees(math.asin(math.sin(yaw)))
        
        # Append landmarks into data
        data.append((shape))
        labels.append([i])
        rotation_degree.append((pitch,roll,yaw))
        count += 1
        sec += 1
#         print(count)
        # print ("Camera Matrix :\n {0}".format(camera_matrix))
        # print ("Rotation Vector:\n {0}".format(rotation_vector))
        # print ("Translation Vector:\n {0}".format(translation_vector))
        # print ("Pitch: {0}\nRoll: {1}\nYaw: {2}".format(pitch, roll, yaw))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()
print("--- %s seconds ---" % (time.time() - start_time))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [3]:
# ''' This function calculates eye aspect ratio using 2D facial landmarks'''
def eye_aspect_ratio(landmarks):
	A = distance.euclidean(landmarks[1], landmarks[5])
	B = distance.euclidean(landmarks[2], landmarks[4])
	C = distance.euclidean(landmarks[0], landmarks[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [4]:
def mouth_aspect_ratio(landmarks):
    ''' This function calculates mouth aspect ratio using 2D facial landmarks'''
    A = distance.euclidean(landmarks[14], landmarks[18])
    C = distance.euclidean(landmarks[12], landmarks[16])
    mar = (A ) / (C)
    return mar

In [5]:
def mouth_over_eye(landmarks):
    ''' This function divides MAR by EAR to get MOE'''
    ear = eye_aspect_ratio(landmarks)
    mar = mouth_aspect_ratio(landmarks)
    mouth_eye = mar/ear
    return mouth_eye

In [6]:
def circularity(landmarks):
    ''' This function calculates Pupil circularity using 2D facial landmarks'''
    A = distance.euclidean(landmarks[1], landmarks[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(landmarks[0], landmarks[1])
    p += distance.euclidean(landmarks[1], landmarks[2])
    p += distance.euclidean(landmarks[2], landmarks[3])
    p += distance.euclidean(landmarks[3], landmarks[4])
    p += distance.euclidean(landmarks[4], landmarks[5])
    p += distance.euclidean(landmarks[5], landmarks[0])
    return 4 * math.pi * Area /(p**2)

In [7]:
# convert landmarks in numpy arrays
data = np.array(data)
rotation_degree = np.array(rotation_degree)
labels = np.array(labels)

In [8]:
# computes facial featuer matrix using defined functions
features = []
for d in data:
    eye = d[36:68]
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    cir = circularity(eye)
    mouth_eye = mouth_over_eye(eye)
    features.append([ear, mar, cir, mouth_eye])

In [9]:
#  concat facial features with euler angles
features = np.array(features)
features = np.concatenate((features,rotation_degree),axis=1)
# check final shape
print(features.shape)
print(labels.shape)

(300, 7)
(300, 1)


In [10]:
# save and validate final data
np.savetxt(r'C:\Users\Pradeep\Downloads\Colab\Data\Alert\40.csv', features, delimiter=',')
df = pd.read_csv(r'C:\Users\Pradeep\Downloads\Colab\Data\Alert\40.csv', header= None)
df.columns = ['EAR', 'MAR', 'CIR', 'MOE', 'PITCH', 'ROLL', 'YAW']
print(df.shape)
df.head(5)


(300, 7)


,EAR,MAR,CIR,MOE,PITCH,ROLL,YAW
0,0.384553,1.029428,0.520889,2.676948,-17.622569,0.124975,1.558267
1,0.277162,0.996678,0.454352,3.596012,-17.090641,0.249736,1.113920
2,0.348173,0.947408,0.491498,2.721083,-18.045332,-0.119269,-1.327842
3,0.354465,1.035930,0.509787,2.922513,-17.224984,-0.697849,2.154456
4,0.370117,1.099013,0.517786,2.969370,-16.772361,1.770658,6.734095
